# Bonsai in Tensorflow

This is a simple notebook that illustrates the usage of Tensorflow implementation of Bonsai. We are using the USPS dataset. Please refer to `fetch_usps.py` and run it for downloading and cleaning up the dataset.

In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

import tensorflow as tf
import numpy as np
import sys
import os

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#Bonsai imports
from edgeml_tf.trainer.bonsaiTrainer import BonsaiTrainer
from edgeml_tf.graph.bonsai import Bonsai

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

sys.path.append(r"E:\programming\practice\research\EdgeML\examples\tf\Bonsai")
import helpermethods

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix,classification_report
from functools import partial

# USPS Data

It is assumed that the USPS data has already been downloaded and set up with the help of [fetch_usps.py](fetch_usps.py) and is present in the `./usps10` subdirectory.

In [ ]:
#Loading and Pre-processing dataset for Bonsai
dataDir = r"E:\programming\practice\research\bonsai\experiments"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = helpermethods.preProcessData(dataDir, isRegression=False)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

# Model Parameters

Note that Bonsai is designed for low-memory setting and the best results are obtained when operating in that setting. Use the sparsity, projection dimension and tree depth to vary the model size.

In [ ]:
sigma = 1.0 #Sigmoid parameter for tanh
depth = 3 #Depth of Bonsai Tree
projectionDimension = 28 #Lower Dimensional space for Bonsai to work on

#Regularizers for Bonsai Parameters
regZ = 0.0001
regW = 0.001
regV = 0.001
regT = 0.001

totalEpochs = 10

learningRate = 0.01

outFile = None

#Sparsity for Bonsai Parameters. x => 100*x % are non-zeros
sparZ = 0.2
sparW = 0.3
sparV = 0.3
sparT = 0.62

batchSize = np.maximum(100, int(np.ceil(np.sqrt(Ytrain.shape[0]))))

useMCHLoss = False #only for Multiclass cases True: Multiclass-Hing Loss, False: Cross Entropy. 

#Bonsai uses one classier for Binary, thus this condition
if numClasses == 2:
    numClasses = 1

Placeholders for Data feeding during training and infernece

In [ ]:
X = tf.placeholder("float32", [None, dataDimension])
Y = tf.placeholder("float32", [None, numClasses])

Creating a directory for current model in the datadirectory using timestamp

In [ ]:
currDir = helpermethods.createTimeStampDir(dataDir)
helpermethods.dumpCommand(sys.argv, currDir)

In [ ]:
X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
def objective(trial,x_train, x_test, y_train, y_test):
    # Inside the optimization function, you use the 'trial' object to suggest hyperparameters
    REG_W = trial.suggest_float('REG_W', 2e-6, 5e-6)
    REG_T = trial.suggest_float('REG_W', 2e-6, 5e-6)
    regV = trial.suggest_float('regV', 2e-5, 5e-5)
    regZ = trial.suggest_float('regZ', 2e-5, 5e-5)
    sparW = trial.suggest_float('sparW', 0.5, 1.0)
    sparB = trial.suggest_float('sparB', 0.5, 1.0)
    sparV = trial.suggest_float('sparV', 0.5, 1.0)
    sparZ = trial.suggest_float('sparZ', 0.5, 1.0)
    depth = trial.suggest_int('depth', 1, 10)
        
    LEARNING_RATE = trial.suggest_float('LEARNING_RATE', 1e-4, 1e-3)
    NUM_EPOCHS = trial.suggest_int('NUM_EPOCHS', 400, 600)

    # Set the suggested hyperparameters in the trainer
    bonsaiObj = Bonsai(numClasses, dataDimension, projectionDimension, depth, sigma)
    
    
    trainer = BonsaiTrainer(bonsaiObj, REG_W, REG_T, regV, regZ, sparW, sparB, sparV, sparZ,
                              LEARNING_RATE, X, Y, useMCHLoss, outFile)
    
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    trainer.train(batchSize, NUM_EPOCHS, sess,x_train, x_test, y_train, y_test, dataDir, currDir)
    pred = bonsaiTrainer.pred1
    f_pred=[]
    for i in pred:
        if i == 0:
            f_pred.append(1)
        else:
            f_pred.append(0)
    sensitivity = confusion_matrix(Ytest,f_pred)[1][1]/(confusion_matrix(Ytest,f_pred)[1][1] + confusion_matrix(Ytest,f_pred)[1][0])
    specificity = confusion_matrix(Ytest,f_pred)[0][0]/(confusion_matrix(Ytest,f_pred)[0][0] + confusion_matrix(Ytest,f_pred)[0][1])
    mcc = matthews_corrcoef(y_test, f_pred)
    return mcc + sensitivity + specificity

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')

In [ ]:
op_fun = partial(objective,x_train=Xtrain, x_test=Xtest, y_train=Ytrain, y_test=Ytest)
study.optimize(op_fun,n_trials=10)

# Bonsai Graph Object

Instantiating the Bonsai Graph which will be used for training and inference.

In [ ]:
bonsaiObj = Bonsai(numClasses, dataDimension, projectionDimension, depth, sigma)

# Bonsai Trainer Object

Instantiating the Bonsai Trainer which will be used for 3 phase training.

In [ ]:
bonsaiTrainer = BonsaiTrainer(bonsaiObj, regW, regT, regV, regZ, sparW, sparT, sparV, sparZ,
                              learningRate, X, Y, useMCHLoss, outFile)

Session declaration and variable initialization. 
Interactive Session doesn't clog the entire GPU.

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Bonsai Training Routine

The method to to run the 3 phase training, followed by giving out the best early stopping model, accuracy along with saving of the parameters.

In [ ]:
bonsaiTrainer.train(batchSize, totalEpochs, sess,
                    Xtrain, Xtest, Ytrain, Ytest, dataDir, currDir)

In [ ]:
pred = bonsaiTrainer.pred1
y_pred=[]
for i in pred:
    if i == 0:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
y_pred

In [ ]:
pred = bonsaiTrainer.pred1
f_pred=[]
for i in pred:
    if i == 0:
        f_pred.append(1)
    else:
        f_pred.append(0)
sensitivity = confusion_matrix(Ytest,f_pred)[1][1]/(confusion_matrix(Ytest,f_pred)[1][1] + confusion_matrix(Ytest,f_pred)[1][0])
specificity = confusion_matrix(Ytest,f_pred)[0][0]/(confusion_matrix(Ytest,f_pred)[0][0] + confusion_matrix(Ytest,f_pred)[0][1])
print(f"sensitivity: {sensitivity} specificity: {specificity}")

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print (confusion_matrix(Ytest,y_pred))
print (classification_report(Ytest,y_pred,digits=5))

In [ ]:
bonsaiTrainer.sigmaI

In [ ]:
import os
os.getcwd()